In [ ]:
from model.pdf import Pdf
from model.columns_pdf import ColumnsPdf
import pdfplumber
from pprint import pprint

In [ ]:
with pdfplumber.open('CDMATR~1.PDF') as pdf:
    page = pdf.pages[0]
    texts = list(filter(lambda i: i, (map(lambda it: j.split('\n') if (j := it.extract_text()) else None, pdf.pages))))

    table = page.extract_table(table_settings={"vertical_strategy": "lines", 
                                         "horizontal_strategy": "text", 
                                         "snap_tolerance": 2,})
    
pdf = Pdf()
columns_pdf = ColumnsPdf()

In [100]:
list_lines = []

cont_exist = False

list_exists = []
for line in table:
    print("\033[93m line =\033[0m", line)
    for text in texts[0][21:]:
        if line[0] in text:
            if line[0] in list_exists:
                cont_exist = True
            else:
                cont_exist = False

            if cont_exist:
                list_exists.clear()
                continue

            list_exists.append(line[0])
            
            # print("\033[96m text_finded =\033[0m", text)
            print("\033[92m finded =\033[0m", text.split())
            columns_pdf.cci = text.split()[0] if text.split()[0].isdigit() else ''
            columns_pdf.descricao = line[0]
            columns_pdf.leitura_alterior = line[1]
            columns_pdf.leitura_atual = line[2]
            columns_pdf.registrado = line[3]
            columns_pdf.faturado = line[4]
            columns_pdf.tarifa_com_icms = line[5]
            columns_pdf.base_icms = line[6]
            columns_pdf.aliq_icms = line[7]
            columns_pdf.icms = line[8]
            columns_pdf.valor = line[9]
            # columns_pdf.tarifa_sem_impostos = line[10]
            # pprint(columns_pdf.to_string())

            pprint(text.split()[-1] == line[-1])
            list_lines.append(columns_pdf)
            break
pdf.tabela = list_lines


 line = ['MEDIDOR N° 15159341', '', '', '', '', '', '', '', '', '']
 finded = ['MEDIDOR', 'N°', '15159341']
False
 line = ['CONST. POTENCIA 0,84000', '', '', '', '', '', '', '', '', '']
 finded = ['CONST.', 'POTENCIA', '0,84000']
False
 line = ['CONST. ATIVO    0,21000', '', '', '', '', '', '', '', '', '']
 finded = ['CONST.', 'ATIVO', '0,21000']
False
 line = ['CONST. REATIVO  0,21000', '', '', '', '', '', '', '', '', '']
 finded = ['CONST.', 'REATIVO', '0,21000']
False
 line = ['CONSUMO PONTA VM', '8.718.104', '8.872.295', '32.380,1', '', '', '', '', '', '']
 finded = ['CONSUMO', 'PONTA', 'VM', '8.718.104', '8.872.295', '32.380,1']
False
 line = ['CONSUMO FORA PONTA CAPACIT. VM', '19.349.389', '19.652.495', '63.652,3', '', '', '', '', '', '']
 finded = ['CONSUMO', 'FORA', 'PONTA', 'CAPACIT.', 'VM', '19.349.389', '19.652.495', '63.652,3']
False
 line = ['CONSUMO FORA PONTA INDUTIVO VM', '70.852.410', '71.939.251', '228.236,6', '', '', '', '', '', '']
 finded = ['CONSUMO', 'FORA', 'PON